# PQL Score - User Preprocessing
This code is used as the preprocessing pipeline for users: 
* users are categorized as generic or commercial 
* only users that are users first are kept (inbound) 

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
u_path = os.path.join("..","data", "raw", "rails","vention_rails_users.csv")
c_path = os.path.join("..","data", "raw","salesforce","vention_salesforce_contact.csv")
gd_path = os.path.join("..","data", "transformed","vention_transformed_salesforce_generic_domains.csv")

df_u = pd.read_csv(u_path)
df_c = pd.read_csv(c_path) 
df_gd = pd.read_csv(gd_path) 

C:\Users\marie-eve.malette_ve\AppData\Local\Temp\ipykernel_11784\3789995714.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_u = pd.read_csv(u_path)
C:\Users\marie-eve.malette_ve\AppData\Local\Temp\ipykernel_11784\3789995714.py:6: DtypeWarning: Columns (16,28,29,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df_c = pd.read_csv(c_path)


In [ ]:
# clean up and formatting
df_u = df_u[['id', 'email', 'created_at', 'activated_datetime']]
df_u['created_at'] = pd.to_datetime(df_u['created_at'], format='%Y-%m-%d %H:%M:%S')
df_u['activated_datetime'] = pd.to_datetime(df_u['activated_datetime'], format='%Y-%m-%d %H:%M:%S')
df_u['id'] = df_u['id'].astype(str)

# keep only 2022 activation period
df_u = df_u.loc[df_u['created_at'] >= pd.to_datetime('2022-01-01')]
df_u = df_u.loc[df_u['activated_datetime'] <= pd.to_datetime('2022-12-31')]

# add generic and commercial flags
generic_domains_list = df_gd['split_part'].tolist()
df_u['domain'] = df_u['email'].str.split('@').str[1]
df_u['generic'] = df_u['domain'].apply(lambda domain: 1 if domain in generic_domains_list else 0)
df_u['domain'] = df_u['email'].str.split('@').str[1]
df_u['commercial'] = df_u['domain'].apply(lambda domain: 0 if domain in generic_domains_list else 1)

In [ ]:
# remove null emails and duplicates
df_c['createddate'] = pd.to_datetime(df_c['createddate'], format='%Y-%m-%d %H:%M:%S')
df_c = df_c.loc[~df_c['email'].isnull()]
df_c.drop_duplicates('email', keep='first', inplace=True)

In [ ]:
# merge users and contacts to retrieve sl success 
df_u_c = pd.merge(df_u, df_c, left_on='email', right_on='email', how='left')
df_u_c['sl_success'] = [1 if s else 0 for s in df_u_c['salesloft_success__c']]

# keep only inbound
df_u_c_ = df_u_c.loc[df_u_c['created_at'] < df_u_c['createddate']]

# remove vention users
df_u_c_ = df_u_c_.loc[~df_u_c_['domain'].str.contains('vention')]

# keep columns to a minimum
df_u_c_final = df_u_c_[['id_x', 'activated_datetime','commercial',  'sl_success']]

df_u_c_final.rename(columns={'id_x': 'user_id',  
                    'activated_datetime': 'user_activated_at'}, inplace=True)

C:\Users\marie-eve.malette_ve\AppData\Local\Temp\ipykernel_11784\1405637214.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_u_c_final.rename(columns={'id_x': 'user_id',


In [ ]:
df_u_c_final.to_csv('tmp/df_users_contacts_final.csv')